# Title/Introduction

## Business Understanding 

## Data Understanding

In [70]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/ibtracs.since1980.list.v04r00.csv', dtype='object', parse_dates=True, skiprows=[1])
#drop first row as it's a multi index

pd.set_option('display.max_columns', None)
df.head(3)

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,WMO_AGENCY,TRACK_TYPE,DIST2LAND,LANDFALL,IFLAG,USA_AGENCY,USA_ATCF_ID,USA_LAT,USA_LON,USA_RECORD,USA_STATUS,USA_WIND,USA_PRES,USA_SSHS,USA_R34_NE,USA_R34_SE,USA_R34_SW,USA_R34_NW,USA_R50_NE,USA_R50_SE,USA_R50_SW,USA_R50_NW,USA_R64_NE,USA_R64_SE,USA_R64_SW,USA_R64_NW,USA_POCI,USA_ROCI,USA_RMW,USA_EYE,TOKYO_LAT,TOKYO_LON,TOKYO_GRADE,TOKYO_WIND,TOKYO_PRES,TOKYO_R50_DIR,TOKYO_R50_LONG,TOKYO_R50_SHORT,TOKYO_R30_DIR,TOKYO_R30_LONG,TOKYO_R30_SHORT,TOKYO_LAND,CMA_LAT,CMA_LON,CMA_CAT,CMA_WIND,CMA_PRES,HKO_LAT,HKO_LON,HKO_CAT,HKO_WIND,HKO_PRES,NEWDELHI_LAT,NEWDELHI_LON,NEWDELHI_GRADE,NEWDELHI_WIND,NEWDELHI_PRES,NEWDELHI_CI,NEWDELHI_DP,NEWDELHI_POCI,REUNION_LAT,REUNION_LON,REUNION_TYPE,REUNION_WIND,REUNION_PRES,REUNION_TNUM,REUNION_CI,REUNION_RMW,REUNION_R34_NE,REUNION_R34_SE,REUNION_R34_SW,REUNION_R34_NW,REUNION_R50_NE,REUNION_R50_SE,REUNION_R50_SW,REUNION_R50_NW,REUNION_R64_NE,REUNION_R64_SE,REUNION_R64_SW,REUNION_R64_NW,BOM_LAT,BOM_LON,BOM_TYPE,BOM_WIND,BOM_PRES,BOM_TNUM,BOM_CI,BOM_RMW,BOM_R34_NE,BOM_R34_SE,BOM_R34_SW,BOM_R34_NW,BOM_R50_NE,BOM_R50_SE,BOM_R50_SW,BOM_R50_NW,BOM_R64_NE,BOM_R64_SE,BOM_R64_SW,BOM_R64_NW,BOM_ROCI,BOM_POCI,BOM_EYE,BOM_POS_METHOD,BOM_PRES_METHOD,NADI_LAT,NADI_LON,NADI_CAT,NADI_WIND,NADI_PRES,WELLINGTON_LAT,WELLINGTON_LON,WELLINGTON_WIND,WELLINGTON_PRES,DS824_LAT,DS824_LON,DS824_STAGE,DS824_WIND,DS824_PRES,TD9636_LAT,TD9636_LON,TD9636_STAGE,TD9636_WIND,TD9636_PRES,TD9635_LAT,TD9635_LON,TD9635_WIND,TD9635_PRES,TD9635_ROCI,NEUMANN_LAT,NEUMANN_LON,NEUMANN_CLASS,NEUMANN_WIND,NEUMANN_PRES,MLC_LAT,MLC_LON,MLC_CLASS,MLC_WIND,MLC_PRES,USA_GUST,BOM_GUST,BOM_GUST_PER,REUNION_GUST,REUNION_GUST_PER,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR
0,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 00:00:00,TS,-12.5000,172.500,,,,main,647,647,O________OO_O_,jtwc_sh,SH051980,-12.5000,172.500,,,25,,-1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-12.5000,172.500,TC,25,,-12.5000,172.500,1,25,,,,,,,-12.5000,172.500,TC,25,,,,,,,,,,,,,,,,,6,351
1,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 03:00:00,TS,-12.1927,172.441,,,,main,653,653,P________PP_P_,,SH051980,-12.1825,172.432,,,25,,-1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-12.1825,172.432,TC,25,,-12.2234,172.469,1,,,,,,,,-12.1825,172.432,TC,25,,,,,,,,,,,,,,,,,6,351
2,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 06:00:00,TS,-11.9144,172.412,,,,main,670,670,O________OP_O_,jtwc_sh,SH051980,-11.9000,172.400,,,25,,-1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-11.9000,172.400,TC,25,,-11.9575,172.450,1,,,,,,,,-11.9000,172.400,TC,25,,,,,,,,,,,,,,,,,5,358


The size of the file is really large but it will get smaller throughout the cleaning process. To start off with, there are 163 columns and they are all reading in as object datatypes. I'll need to go through and clean these up.

In [3]:
df.shape

(271883, 163)

In [4]:
df.columns = [x.lower() for x in df.columns]
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271883 entries, 0 to 271882
Data columns (total 163 columns):
sid                 object
season              object
number              object
basin               object
subbasin            object
name                object
iso_time            object
nature              object
lat                 object
lon                 object
wmo_wind            object
wmo_pres            object
wmo_agency          object
track_type          object
dist2land           object
landfall            object
iflag               object
usa_agency          object
usa_atcf_id         object
usa_lat             object
usa_lon             object
usa_record          object
usa_status          object
usa_wind            object
usa_pres            object
usa_sshs            object
usa_r34_ne          object
usa_r34_se          object
usa_r34_sw          object
usa_r34_nw          object
usa_r50_ne          object
usa_r50_se          object
usa_r50_sw          obje

The dataset has readings for storms at multiple points in their progression. There are 4,458 unique storms tracked.

In [5]:
#there are this many separate storms
df['sid'].nunique()

4458

My classification task will be to identify whether they are minor storms or severe Tropical Storms. Looking at my target column, nature, I can see six different classes that I want to sort into two so this will be a binary - severe storm or not severe. 

NR, not reported, and MX, mixture will be removed as they don't tell me anything. TS, tropical storm, will be my '1' - a severe storm. ET, DS, and SS are extratropical, disturbance, and subtropical - less severe storms. These will be my '0' class. 

## Data Exploration & Cleaning

In [6]:
df['nature'].unique()

array(['TS', 'NR', 'ET', 'MX', 'SS', 'DS'], dtype=object)

In [7]:
df.drop(df.loc[df['nature'] == 'NR'].index, inplace=True)
df.drop(df.loc[df['nature'] == 'MX'].index, inplace=True)

In [8]:
#new column  
df['target'] = 0

# loop through the data and input a 1 where the storm is a Tropical storm
for row in df.index:
    if df['nature'][row] == 'TS':
        df['target'][row] = 1

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
df['target'].value_counts(normalize=True)

1    0.897215
0    0.102785
Name: target, dtype: float64

So there is a pretty severe class imbalance here. Before I address that, I want to define my features. Taking a preliminary look through the columns in the dataframe and the documentation as a guide, I'm selecting the following as potential features to explore. This is a lot less than 163! A lot of these were blank, repeats, or not useful info.

In [10]:
initial_feats = ['sid', 'season', 'basin', 'subbasin', 'iso_time', 'lat', 'lon', 
                 'wmo_wind', 'dist2land', 'ds824_wind', 'td9636_stage', 'storm_speed', 'storm_dir']

In [11]:
xs_df = df[initial_feats]
xs_df.head()

,sid,season,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir
0,1980001S13173,1980,SP,MM,1980-01-01 00:00:00,-12.5000,172.500,,647,25,1,6,351
1,1980001S13173,1980,SP,MM,1980-01-01 03:00:00,-12.1927,172.441,,653,25,1,6,351
2,1980001S13173,1980,SP,MM,1980-01-01 06:00:00,-11.9144,172.412,,670,25,1,5,358
3,1980001S13173,1980,SP,MM,1980-01-01 09:00:00,-11.6863,172.435,,682,25,1,4,12
4,1980001S13173,1980,SP,MM,1980-01-01 12:00:00,-11.5000,172.500,,703,25,1,4,22


I'm able to convert some of the columns to workable datatypes. Some, that I've listed as 'remaining columns' ran an error as they have empty strings. For now I'm filling those with an actual null so that I am able to work with them.

In [12]:
xs_df[['lat', 'lon', 'dist2land']] = xs_df[['lat', 'lon', 'dist2land']].apply(pd.to_numeric)
xs_df[['iso_time', 'season']] = xs_df[['iso_time', 'season']].apply(pd.to_datetime)
xs_df['season'] = xs_df['season'].dt.year

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
remaining_cols = ['wmo_wind', 'ds824_wind', 'td9636_stage', 'storm_speed', 'storm_dir']
xs_df[remaining_cols] = xs_df[remaining_cols].apply(lambda x: x.replace(' ', None))
xs_df[remaining_cols] = xs_df[remaining_cols].apply(pd.to_numeric)

In [33]:
xs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226423 entries, 0 to 271072
Data columns (total 13 columns):
sid             226423 non-null object
season          226423 non-null int64
basin           190696 non-null object
subbasin        197687 non-null object
iso_time        226423 non-null datetime64[ns]
lat             226423 non-null float64
lon             226423 non-null float64
wmo_wind        226423 non-null int64
dist2land       226423 non-null int64
ds824_wind      226423 non-null int64
td9636_stage    226423 non-null int64
storm_speed     226423 non-null int64
storm_dir       226423 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(7), object(3)
memory usage: 34.2+ MB


In [34]:
y_df = df['target'].to_frame()
clean_df = pd.concat([xs_df, y_df], axis=1)

,sid,season,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir,target
0,1980001S13173,1980,SP,MM,1980-01-01 00:00:00,-12.5000,172.500,0,647,25,1,6,351,1
1,1980001S13173,1980,SP,MM,1980-01-01 03:00:00,-12.1927,172.441,0,653,25,1,6,351,1
2,1980001S13173,1980,SP,MM,1980-01-01 06:00:00,-11.9144,172.412,0,670,25,1,5,358,1
3,1980001S13173,1980,SP,MM,1980-01-01 09:00:00,-11.6863,172.435,0,682,25,1,4,12,1
4,1980001S13173,1980,SP,MM,1980-01-01 12:00:00,-11.5000,172.500,0,703,25,1,4,22,1


All of my non-object columns have been converted! There are still null values in the basin & subbasin columns so I'm going to inspect those to see what the best action would be.

In [35]:
grouped_df = clean_df.groupby(['sid']).mean()

1.000000    2477
0.000000      21
0.666667      20
0.800000      16
0.857143      12
            ... 
0.868421       1
0.275862       1
0.991304       1
0.764706       1
0.907407       1
Name: target, Length: 857, dtype: int64

From inspecting the mean value for the target column of my data grouped by storm ID, I can see that storms change class over time. Because of this, I'm not going to look at the readings by storm but instead as independent readings to see what conditions change a storm to be severe

In [28]:
clean_df.loc[clean_df['sid'] == '2020307N12250']

,sid,season,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir,target
270814,2020307N12250,2020,EP,MM,2020-11-03 12:00:00,12.7000,-114.500,0,1236,0,0,9,299,0
270815,2020307N12250,2020,EP,MM,2020-11-03 15:00:00,13.1324,-114.962,0,1216,0,0,14,316,0
270816,2020307N12250,2020,EP,MM,2020-11-03 18:00:00,13.7000,-115.500,0,1181,0,0,15,312,1
270817,2020307N12250,2020,EP,MM,2020-11-03 21:00:00,14.1225,-116.077,0,1175,0,0,14,305,1
270818,2020307N12250,2020,EP,MM,2020-11-04 00:00:00,14.5000,-116.700,0,1174,0,0,14,299,1
270819,2020307N12250,2020,EP,MM,2020-11-04 03:00:00,14.8251,-117.373,0,1194,0,0,14,299,1
270820,2020307N12250,2020,EP,MM,2020-11-04 06:00:00,15.2000,-118.000,0,1203,0,0,14,312,1
270821,2020307N12250,2020,EP,MM,2020-11-04 09:00:00,15.7486,-118.443,0,1193,0,0,14,323,1
270822,2020307N12250,2020,EP,MM,2020-11-04 12:00:00,16.3333,-118.900,0,1175,0,0,16,315,1
270823,2020307N12250,2020,EP,MM,2020-11-04 15:00:00,16.8479,-119.595,0,1177,0,0,16,305,1


In [36]:
clean_df = clean_df.drop('sid', axis=1)

In [23]:
clean_df['td9636_stage'].value_counts()

0    186588
2     15923
1     12485
4     10273
3       734
5       228
6       192
Name: td9636_stage, dtype: int64

In [38]:
clean_df.loc[clean_df['td9636_stage'] == 3]

,season,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir,target
2322,1980,EP,MM,1980-06-10 18:00:00,9.9000,-114.900,70,1504,70,3,4,332,1
2323,1980,EP,MM,1980-06-10 21:00:00,10.0366,-115.000,0,1503,70,3,4,316,1
2483,1980,EP,MM,1980-06-26 18:00:00,15.3000,-108.300,65,573,65,3,13,284,1
2484,1980,EP,MM,1980-06-26 21:00:00,15.4423,-108.908,0,608,65,3,12,285,1
3136,1980,EP,MM,1980-07-29 18:00:00,19.9000,-114.500,65,573,65,3,10,300,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66793,1989,WP,MM,1989-11-28 09:00:00,10.6252,140.580,0,1379,0,3,10,290,1
67293,1989,WP,MM,1989-12-24 00:00:00,12.1800,150.520,55,1638,0,3,6,319,1
67294,1989,WP,MM,1989-12-24 03:00:00,12.4053,150.340,0,1655,0,3,6,319,1
67295,1989,WP,MM,1989-12-24 06:00:00,12.5975,150.152,60,1675,0,3,5,306,1


Stage appears to be target leakage but we can see the relationship isn't 1:1 except in one instance. 

In [29]:
grouped_stage = clean_df.groupby(['td9636_stage']).mean()
grouped_stage

,season,lat,lon,wmo_wind,dist2land,ds824_wind,storm_speed,storm_dir,target
td9636_stage,,,,,,,,,
0,2002.277756,10.622475,41.780129,20.261105,827.878636,0.141279,10.158976,211.382902,0.878336
1,1984.771726,2.917810,53.959564,9.228594,834.593192,2.552503,8.974289,225.891870,0.979816
2,1984.469321,3.299528,60.005174,20.196131,847.935691,4.380330,9.586447,222.150411,0.985493
3,1985.912807,10.500353,30.093530,31.164850,925.190736,1.873297,8.931880,260.816076,1.000000
4,1984.644310,8.750767,59.260186,37.466271,877.548915,6.627081,9.714592,227.356760,0.999124
5,1984.267544,10.635196,134.457014,11.732456,647.399123,2.526316,19.258772,96.333333,0.671053
6,1982.541667,16.970099,120.258646,3.750000,349.078125,1.625000,11.421875,212.692708,0.973958


In [30]:
clean_df.groupby(['target']).mean()

,season,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir
target,,,,,,,,,
0,2004.657887,26.183926,14.152830,10.345379,844.710480,0.000000,0.049628,14.850986,148.837451
1,1998.543692,7.703185,48.142032,21.598198,830.143382,0.976224,0.436928,9.488275,221.142289


My remaining string columns, basin and subbasin, have a solid amount of null values. In trying to handle these, I first took a look at min & max lat/long values to see if I could infer from there but there was overlap. Looking at the documentation I can see that MM, the category within subbasin, is missing. Because this is the most popular category, for both I'll try imputing nulls with the most common value as my best guess. 

In [39]:
clean_df['basin'].value_counts()

WP    75404
SI    48893
EP    41571
SP    20576
NI     4133
SA      119
Name: basin, dtype: int64

In [45]:
clean_df['basin'].isna().sum() / len(clean_df['basin'])

0.1577887405431427

In [50]:
clean_df['subbasin'].value_counts()

MM    154639
WA     17735
EA      7245
CP      6944
CS      3733
GM      3258
BB      2476
AS      1657
Name: subbasin, dtype: int64

In [44]:
clean_df['subbasin'].isna().sum() / len(clean_df['subbasin'])

0.12691290195783997

## Modeling

Simply guessing 1 will mean I am accurate ~90% of the time. I want to account for class imbalance when running my first model.

In [55]:
# Your code here
y = clean_df['target']
X =  clean_df.drop(columns=['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Preprocessing for first model

In [67]:
num_cols = []
ohe_cols = []
    
for c in X.columns:
    if df[c].dtype in ['float64', 'int64']:
        num_cols.append(c)
    elif df[c].dtype in ['object']:
        ohe_cols.append(c)

In [68]:
num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

ohe_transformer = Pipeline(steps=[
    ('str_imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='error'))])


In [72]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat_ohe', ohe_transformer, ohe_cols) 
        ])

In [73]:
preprocessor.fit_transform(X_train).shape

(169817, 262201)

In [74]:
clf_logreg = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', LogisticRegression(class_weight='balanced'))])

clf_logreg.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('num_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                         

In [76]:
train_preds = clf_logreg.predict(X_train)
test_preds = clf_logreg.predict(X_test)

ValueError: Found unknown categories [Timestamp('1990-12-05 15:00:00'), Timestamp('1991-12-19 21:00:00'), Timestamp('1984-11-27 09:00:00'), Timestamp('1996-12-28 16:00:00'), Timestamp('2012-01-24 09:00:00'), Timestamp('2016-11-26 18:00:00'), Timestamp('2018-04-07 12:00:00'), Timestamp('1995-01-18 15:00:00'), Timestamp('1986-06-07 03:00:00'), Timestamp('2004-08-29 14:00:00'), Timestamp('1991-07-03 09:00:00'), Timestamp('1994-04-21 21:00:00'), Timestamp('1990-02-13 09:00:00'), Timestamp('2008-05-14 09:00:00'), Timestamp('2008-06-18 12:00:00'), Timestamp('1981-06-25 03:00:00'), Timestamp('1981-07-31 03:00:00'), Timestamp('1992-01-03 06:00:00'), Timestamp('2004-12-15 21:00:00'), Timestamp('2000-10-23 18:00:00'), Timestamp('2007-05-21 21:00:00'), Timestamp('2017-05-11 00:00:00'), Timestamp('2005-05-18 03:00:00'), Timestamp('2003-08-06 21:00:00'), Timestamp('2018-05-25 00:00:00'), Timestamp('1998-11-20 00:00:00'), Timestamp('1986-07-29 00:00:00'), Timestamp('1991-02-02 12:00:00'), Timestamp(

In [ ]:
accuracy_score(y_train, train_preds)

In [ ]:
#date time objects in models